In [84]:
import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from photutils import aperture_photometry, CircularAperture, CircularAnnulus
from astropy.wcs import WCS
import glob



In [85]:
# Define the directory containing the FITS files
file_names = glob.glob('/Users/wendymendoza/Desktop/Yan/2023-03-16/*.fits')
if not file_names:
    print("No files found in the specified directory.")
else:
    # Process each FITS file
    for file_name in file_names:
        with fits.open(file_name) as hdul:
            header = hdul[0].header
            data = hdul[0].data


In [86]:
# Define the target's and reference stars' coordinates (RA, Dec) as tuples
# Example: (RA, Dec)
target_coordinates = (123.456, -12.345)
reference_coordinates = [(124.567, -13.456), (125.678, -14.567), (126.789, -15.678)]

# Convert RA and Dec to pixel coordinates
target_x, target_y = wcs.all_world2pix(target_coordinates[0], target_coordinates[1], 1)
reference_coords_pixel = [wcs.all_world2pix(ra, dec, 1) for ra, dec in reference_coordinates]

# Round the pixel coordinates to integers
target_x, target_y = int(round(float(target_x))), int(round(float(target_y)))
reference_coords_pixel = [(int(round(float(x))), int(round(float(y)))) for x, y in reference_coords_pixel]

# Create the aperture and annulus objects for the target and reference stars
apertures = [CircularAperture(coord, r=r_aperture) for coord in [(target_x, target_y)] + reference_coords_pixel]
annulus = CircularAnnulus((target_x, target_y), r_in=r_in, r_out=r_out)

# ... continue with your processing code


In [99]:
# Define the target's coordinates (x, y) and the aperture/annulus radii
r_aperture = 5
r_in, r_out = 10, 15


In [94]:
# Initialize lists to store time and flux values for the light curve
time_values = []
flux_values = []

In [102]:
for file_name in file_names:
    with fits.open(file_name) as hdul:
        header = hdul[0].header
        data = hdul[0].data
        time = header['DATE-OBS']

        # Calculate the background level using the annulus
        annulus_mask = annulus.to_mask(method='center')
        
        # Corrected code:
        annulus_data = annulus_mask.multiply(data, fill_value=0)
        annulus_data_1d = annulus_data[annulus_mask.data > 0]
        # End of corrected code

        _, median_sigclip, _ = sigma_clipped_stats(annulus_data_1d)

        # Perform aperture photometry
        phot = aperture_photometry(data, aperture)
        phot['annulus_median'] = median_sigclip
        phot['aper_bkg'] = median_sigclip * aperture.area
        phot['aper_sum_bkgsub'] = phot['aperture_sum'] - phot['aper_bkg']
        # Store the time and flux values
        time_values.append(time)
        flux_values.append(phot['aper_sum_bkgsub'][0])

# Normalize the flux values
flux_values = np.array(flux_values, dtype=float)
flux_values /= np.median(flux_values)

# Create the light curve plot
plt.figure(figsize=(10, 6))
plt.plot(time_values, flux_values, 'o-', markersize=4)
plt.xlabel('Time')
plt.ylabel('Normalized Flux')
plt.title('Light Curve')
plt.show()


TypeError: 'NoneType' object is not subscriptable

SINGLE FITS FILE

In [31]:
import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from photutils import aperture_photometry, CircularAperture, CircularAnnulus
from astropy.wcs import WCS


In [30]:
# Open the FITS file
file_name = '/Users/wendymendoza/Desktop/Yan/2023-03-16/stacked_5.fits'
with fits.open(file_name) as hdul:
    header = hdul[0].header
    data = hdul[0].data

In [32]:
# Assuming you have the header of your FITS file
header = hdul[0].header

# Create a WCS object using the header information
wcs = WCS(header)

# Define the target's RA and Dec (in degrees)
ra, dec = 152.9324407, 57.3038326  # Replace these with the actual RA and Dec


In [34]:
# Convert RA and Dec to pixel coordinates
x, y = wcs.all_world2pix(ra, dec, 1)  # 1 is for the origin (FITS-like)

# Convert numpy arrays to float and round the pixel coordinates to integers
x, y = int(round(float(x))), int(round(float(y)))


In [35]:
# Define the target's coordinates (x, y) and the aperture/annulus radii
r_aperture = 5
r_in, r_out = 10, 15


In [36]:
# Create the aperture and annulus objects
aperture = CircularAperture((x, y), r=r_aperture)
annulus = CircularAnnulus((x, y), r_in=r_in, r_out=r_out)

In [37]:
# Calculate the background level using the annulus
annulus_masks = annulus.to_mask(method='center')
annulus_data = annulus_masks.multiply(data)
annulus_data_1d = annulus_data[annulus_masks.data > 0]
_, median_sigclip, _ = sigma_clipped_stats(annulus_data_1d)

# Perform aperture photometry
phot = aperture_photometry(data, aperture)
phot['annulus_median'] = median_sigclip
phot['aper_bkg'] = median_sigclip * aperture.area
phot['aper_sum_bkgsub'] = phot['aperture_sum'] - phot['aper_bkg']

# Print the results
print(phot['aper_sum_bkgsub']) 

 aper_sum_bkgsub  
------------------
-20.59005106832774
